In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
%matplotlib inline
from basic_units import radians  # FIXME: easier to hard code labels
from itertools import product
from matplotlib.ticker import MaxNLocator

from scipy.stats import gaussian_kde

def kde_scipy(x, x_grid, bandwidth=0.02, **kwargs):
    """Kernel Density Estimation with Scipy"""
    # Note that scipy weights its bandwidth by the covariance of the
    # input data.  To make the results comparable to the other methods,
    # we divide the bandwidth by the sample standard deviation here.
    kde = gaussian_kde(x, bw_method=bandwidth / x.std(ddof=1), **kwargs)
    return kde.evaluate(x_grid)

def angle(u, v):
    n = np.sqrt(np.dot(u, u) * np.dot(v, v))
    if n == 0:
#         print 'zero norm'
        return 0
    else: 
        cs = np.dot(u, v) / n
        return np.arccos(cs)

def random_rot(dim):
    """Return a random rotation matrix, drawn from the Haar distribution
    (the only uniform distribution on SO(n)).
    The algorithm is described in the paper
    Stewart, G.W., 'The efficient generation of random orthogonal
    matrices with an application to condition estimators', SIAM Journal
    on Numerical Analysis, 17(3), pp. 403-409, 1980.
    For more information see
    http://en.wikipedia.org/wiki/Orthogonal_matrix#Randomization"""
    H = np.eye(dim)
    D = np.ones((dim,))
    for n in range(1, dim):
        x = np.random.randn(dim-n+1)
        D[n-1] = np.sign(x[0])
        x[0] -= D[n-1]*np.sqrt((x*x).sum())
        # Householder transformation

        Hx = np.eye(dim-n+1) - 2.*np.outer(x, x)/(x*x).sum()
        mat = np.eye(dim)
        mat[n-1:,n-1:] = Hx
        H = np.dot(H, mat)
    # Fix the last sign such that the determinant is 1
    D[-1] = -D.prod()
    H = (D*H.T).T
    return H

def straight_through(x):
    return np.clip(x, -1, 1)

def binarize(x):
    a = -1 * (x < 0)
    b = 0 * (x == 0)
    c = 1 * (x > 0)
    return a + b + c

def ternarize(v, thresh):
    """Ternarize a vector with threshold quant."""
    return (abs(v) > thresh) * np.sign(v)

In [ ]:
xx = np.arange(-2, 2, 0.01)
yy = ternarize(xx, 0.02)
fig, ax = plt.subplots(1, 1)
ax.plot(xx, yy)
a=1.5
ax.set_xlim(-a, a)
ax.set_ylim(-a, a)
ax.set_aspect('equal')
ax.plot(xx, straight_through(xx), c='r')

In [ ]:
def gen_ctns_vec(d, exp=False, rot=None, k=0.2):
#     return np.random.uniform(-1, 1, size=n)
    x = np.random.randn(d)
    if exp:
        exp_ = np.exp(k * (-d/2. + np.arange(0, d, 1)))
        x = np.multiply(x, exp_)
    if rot is not None:
        assert rot.shape == (d, d)
        x = np.dot(x, rot)
    return x

def ctns_ctns_pair_angle(d, exp=False, rot=None):
    u, v = [gen_ctns_vec(d, exp=exp, rot=rot) for _ in range(2)]
    return angle(v, u)


def ctns_bin_self_angle(d, exp=False, rot=None, k=None):
    u = gen_ctns_vec(d, exp=exp, rot=rot, k=k)
    return angle(binarize(u), u)


def ctns_tern_self_angle(d, thresh=2):
    u = gen_ctns_vec(d, exp=False, rot=None, k=None)
    return angle(ternarize(u, thresh=thresh), u)

In [ ]:
def cc_cb_plot(fig, ax, cc, cb, legend=False, 
               a=np.arccos(np.sqrt(2/np.pi))
              ):
    """
    cc: array, shape (n_s,)
        Samples of the angle between two continuous vectos
    cb: array, shape (n_s,)
        Samples of the angle between a vector and its binarized version
    """
    x_grid = np.arange(0, np.pi, 0.01)
    cc_pdf = kde_scipy(cc, x_grid)
    cb_pdf = kde_scipy(cb, x_grid)
    x_grid_rad = [u * radians for u in x_grid]
    for pdf, label, c in zip(
        [cc_pdf, cb_pdf],
        [r'$\angle [u, v]$', r'$\angle [u, \theta (u)]$'],
        ['b', 'r']):
        ax.plot(x_grid_rad, pdf, alpha=1.0, lw=3, xunits=radians, label=label, c=c, linestyle='-')    

    if a is not None:
        ax.axvline(x=a * radians, c='black', linestyle='--', lw=2)
    
#     ax.set_xlabel('Angle (rad)')
    ax.set_xlabel('')
    ax.get_yaxis().set_visible(False)
    if legend:
        ax.legend(loc='best', fontsize=8, borderpad=0.2)
    return x_grid, cc_pdf, cb_pdf

In [ ]:
def exp_rot_plot(fig, ax, cb, cbr, legend=False):
    """
    cb: array, shape (n_s,)
        Samples of the angle between a vector and its binarized version
    cbr: array, shape (n_s,)
        Samples of the angle between a vector and random rotation binarized version
    """
    x_grid = np.arange(0, np.pi, 0.01)
    cb_pdf = kde_scipy(cb, x_grid)
    cbr_pdf = kde_scipy(cbr, x_grid)
    x_grid_rad = [u * radians for u in x_grid]
    for pdf, label, c in zip(
        [cb_pdf, cbr_pdf],
        [r'$\angle [u, \theta(u)]$', r'$\angle [u, \theta_R (u)]$'],
        ['r', 'orange']):
        ax.plot(x_grid_rad, pdf, alpha=1.0, lw=3, xunits=radians, label=label, c=c, linestyle='-')    

    a = np.arccos(np.sqrt(2/np.pi))
    ax.axvline(x=a * radians, c='black', linestyle='--', lw=2)
    
#     ax.set_xlabel('Angle (rad)')
    ax.set_xlabel('')
    ax.get_yaxis().set_visible(False)
    if legend:
        ax.legend(loc='best', fontsize=8, borderpad=0.2)

# Binary Network Analysis

In [ ]:
n_s = 10000
d_ = [3, 10, 27]

cc_ = {}
cb_ = {}
for d in d_:
#     rot = random_rot(d)
    rot = None
    cc_[d] = np.array([ctns_ctns_pair_angle(d, exp=False, rot=rot) for _ in range(n_s)])
    cb_[d] = np.array([ctns_bin_self_angle(d, exp=False, rot=rot) for _ in range(n_s)])

In [ ]:
fig, axes = plt.subplots(1, 3, sharey=True, figsize=(4, 2.0))
fig.subplots_adjust(wspace=0.2)
for i, d in enumerate(d_):
    cc = cc_[d]
    cb = cb_[d]
    ax = axes[i]
    ax.set_title('d = {}'.format(d))
    cc_cb_plot(fig, ax, cc, cb, legend=i==0, a=None)
#plt.savefig('ctns_rand_bin.pdf', dpi=200)

In [ ]:
fig, axes = plt.subplots(1, 1, sharey=True, figsize=(2, 2))
fig.subplots_adjust(wspace=0.2)
d = d_[-1]
cc = cc_[d]
cb = cb_[d]
ax.set_title('d = {}'.format(d))
_ = cc_cb_plot(fig, axes, cc, cb, legend=True)
#plt.savefig('ctns_rand_bin25.png', dpi=200)

# Exponential Covariance Matrix with and without Rotation

In [ ]:
n_s = 100
d_ = [10, 100, 500]

k = 0.1

cb_ = {}
cbr_ = {}
for d in d_:
    rot = random_rot(d)
    cb_[d] = np.array([ctns_bin_self_angle(d, exp=True, rot=None, k=k) for _ in range(n_s)])
    cbr_[d] = np.array([ctns_bin_self_angle(d, exp=True, rot=rot, k=k) for _ in range(n_s)])

In [ ]:
fig, axes = plt.subplots(1, 3, sharey=True, figsize=(4, 2.0))
fig.subplots_adjust(wspace=0.2)
for i, d in enumerate(d_):
    ax = axes[i]
    ax.set_title('d = {}'.format(d))
    exp_rot_plot(fig, ax, cb_[d], cbr_[d], legend=i==0)
plt.savefig('exp_rot_bin.pdf', dpi=200)

# Higher Order Quantization

In [ ]:
n_s = 10000
d_ = [1000]

emp = 0.02 / 0.18
print emp  # empirical value threshold / sigma

thresh_ = [0.01, emp, 0.2, 0.3, 0.5]
cc_ = []
cb_ = []
for d, thresh in product(d_, thresh_):
    rot = None
    cc_angles = np.array([ctns_ctns_pair_angle(d) for _ in range(n_s)])
    cb_angles = np.array([ctns_tern_self_angle(d, thresh=thresh) for _ in range(n_s)])
    cc_.append((d, thresh, cc_angles))
    cb_.append((d, thresh, cb_angles))

In [ ]:
def f(x):
    return (x[0], x[1], np.median(x[2]))

peaks = map(f, cb_)
dd_, qq_, yy = zip(*peaks)

fig, ax = plt.subplots(figsize=(2, 2))
ax.plot(qq_, yy, 'o-')
ax.set_ylim([0, np.pi/4])
ax.set_yticks([0, np.pi / 8, np.pi/4])
ax.set_yticklabels([0, r'$\pi/8$', r'$\pi/4$'])
ax.set_xlim([0, 0.6])
ax.xaxis.set_major_locator(MaxNLocator(2))

ax.set_title('Peak Angle\n vs Threshold')
plt.tight_layout()
plt.savefig('peak_angle_vs_threshold_tern.pdf')
exp = yy[np.where(np.array(qq_) == emp)[0][0]]
print exp * 180. / np.pi  # Peak value as a function of threshold

In [ ]:
fig, ax = plt.subplots(figsize=(2, 2))

for cc, cb in zip(cc_, cb_):
    d, quant, cc = cc
    d, quant, cb = cb
    if quant != emp:
        continue
    cc_cb_plot(fig, ax, cc, cb, legend=True, a=exp)

ax.set_title('d = {}'.format(d))
plt.savefig('ternary_angle_dist.pdf')

In [ ]:
norm.cdf(emp) - norm.cdf(-emp)  # Fraction of 0 weights assuming a normal distribution. 

# Approximate Backprop

In [ ]:
fig, ax = plt.subplots(1, 2, sharey=True, figsize=(2.6, 1.6))
fig.subplots_adjust(wspace=0.2)

a = 2.3
x = np.arange(-a, a, 0.01)
titles = ['$f^{forward}(x)$', '$f^{backward}(x)$']
ys = [binarize(x), straight_through(x)]

for i, (y, title) in enumerate(zip(ys, titles)):
    ax[i].set_ylim([-a, a]);
    ax[i].set_xlim([-a, a]);
    ax[i].plot(x, y, lw=3, alpha=0.5)

    ax[i].set_aspect('equal')
    ax[i].set_title(title)

plt.tight_layout()
plt.savefig('straight_through.pdf')

# Binary random vectors (Outdated)

In [ ]:
n = 10
n_s = 5000

In [ ]:
def gen_bin_vec(n):
    return np.random.randint(2, size=n) * 2 - 1

def gen_angle(n):
    u, v = [gen_bin_vec(n) for _ in range(2)]
    return angle(u, v)

In [ ]:
data = []
n_ = [10, 100, 1000]
for n in n_:
    data.append((n, np.array([gen_angle(n) for _ in range(n_s)])))

In [ ]:
fig, ax = plt.subplots(1, 3, sharey=True, figsize=(6, 2))
fig.subplots_adjust(wspace=0.2)
for i, (k, v) in enumerate(data):
    x_grid = np.arange(0, np.pi, 0.01)
    pdf = kde_scipy(v, x_grid)
    ax[i].plot(
        [u * radians for u in x_grid], 
         pdf, alpha=0.5, lw=3, xunits=radians)    
    ax[i].set_xlabel('Angle (rad)')
    ax[i].get_yaxis().set_visible(False)
    ax[i].set_title('n = {}'.format(k))
#     break
plt.savefig('hd_angles.pdf', dpi=200)